# Make freeview images

- make freeview images, adding angle jitter  
- several jitters per subject
- currently settings: left hemisphere, ventral view, inflated (fsaverage) base, curv or thickness overlay
- expects a folder in working directory called ./data/thickness or ./data/curv

In [1]:
import numpy as np
import os
import sys

In [2]:
# specify location of data
# expected format is .mgh, in a subdirectory labeled thickness of curv
data_dir = os.getcwd()

In [3]:
data_dir

'/Users/samjohns/fsa-data'

In [4]:
os.chdir(data_dir)

In [5]:
# directory structure: data directory contains subdirs thickness and curv

thickness_subdir = '/thickness'
thickness_dir = data_dir + thickness_subdir

curv_subdir = '/curv'
curv_dir = data_dir + curv_subdir

In [6]:
image_subdir = '/images/images-curv-sphere'
image_dir = data_dir + image_subdir

if not os.path.exists(image_dir):
    os.mkdir(image_dir)

In [7]:
# default values
SCALE=10
VIEWS=10
CENTER=np.array([-65, 0, 0])

def generate_angles(nviews=VIEWS, scale=SCALE, center=CENTER):
    center = np.array(center) # cast to np.array for broadcasting addition
    angle_jitters = scale * (np.array(np.split(np.random.rand(3 * nviews), nviews)) - 0.5)
    return center + angle_jitters

In [8]:
def make_command_strings(input_file_name, pic_base_name, angles, base='sphere', feature='curv'):
    # different settings for curv and sphere base
    if base=='inflated':
        zoom=1.8
        center_ras='-45 0 0'
    elif base=='sphere':
        zoom=1.2
        center_ras='0 0 0'
    command_strings = []
    
    if feature=='curv':
        subdir=curv_subdir
        overlay_settings='overlay_custom=./curv/custom_grayscale'
    elif feature=='thickness':
        subdir=thickness_subdir
        overlay_settings='overlay_threshold=0,4.5'
    for i, a in enumerate(angles):
        command_strings.append(f'freeview -f .{subdir}/lh.sphere:curvature_method="binary":overlay=.{curv_subdir}/{input_file_name}:{overlay_settings} -cam dolly {zoom} elevation {a[0]:.2f} azimuth {a[1]:.2f} roll {a[2]:.2f} -ras {center_ras} -cc -ss .{image_subdir}/{pic_base_name}-{a[0]:.2f}-{a[1]:.2f}-{a[2]:.2f}.png')
    return command_strings

In [9]:
# process input filenames; filter out other directory entries; strip .mgh extension
input_files = [f[:-4] for f in os.listdir(curv_dir) if 'sub-' in f and '.mgh' in f]

In [10]:
input_files[:5], len(input_files)

(['sub-NDARGT022BEW.fsa.lh.curv',
  'sub-NDARTU768MY1.fsa.lh.curv',
  'sub-NDARDN229AKK.fsa.lh.curv',
  'sub-NDARDD073JKZ.fsa.lh.curv',
  'sub-NDARJU903AZH.fsa.lh.curv'],
 2388)

In [11]:
def make_subject_images(input_file, base='sphere'):
    angles = generate_angles()
    command_strings = make_command_strings(input_file+'.mgh', input_file, angles, base=base)
    for command_string in command_strings:
        os.system(command_string)
        
        ## debugging ##
        #print(command_string)

In [12]:
os.getcwd()

'/Users/samjohns/fsa-data'

In [ ]:
# this cell makes the images
# it can take a long time (~ 2 - 10s per image)

# resume from last time
for input_file in input_files[:2231]:
    make_subject_images(input_file, base='sphere')

In [ ]:
31 roll 0.71 -ras -45 0 0 -cc -ss test.png

In [ ]:
! freeview -f ./lh.inflated:curvature_method='binary':overlay=./lh.thickness -cam dolly 1.5 elevation 65 azimuth 0 roll 0 -ss test6500.png